In [20]:
from collections import defaultdict, OrderedDict
from csv import reader
from itertools import chain, combinations
from optparse import OptionParser

In [21]:
class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        ############################ 修改部分 ############################
        # 完成 display() 函數的輸出功能，可以查看 FP-Tree 的結構
        print ('   '*(ind-1), '└─', self.itemName, '(', self.count,')')                  
        ##################################################################
        for child in list(self.children.values()):
            child.display(ind+1)

In [22]:
def constructTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    # Deleting items below minSup
    headerTable = dict((item, sup) for item, sup in headerTable.items() if minSup <= sup )
    headerTable = {k: v for k, v in sorted(headerTable.items(), key=lambda item: (-item[1],item[0]))}

    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    fpTree = Node('Null', 1, None)

    # Update FP tree for each cleaned and sorted itemSet
    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        ############################ 修改部分 ##################################
        # 改成 reverse = True，讓 item 依照 Support 大到小去進行排
        itemSet.sort(key=lambda item: (headerTable[item][0],item), reverse=True)
        #######################################################################
        #print(itemSet)
        # Traverse from root to leaf, update tree with given item
        currentNode = fpTree
        for item in itemSet:
            currentNode = updateTree(item, currentNode, headerTable, frequency[idx])

    #fpTree.display()
    #checkNodeLinks(headerTable)
    return fpTree, headerTable

def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        # If the item already exists, increment the count
        treeNode.children[item].increment(frequency)
    else:
        # Create a new branch
        newItemNode = Node(item, frequency, treeNode)
        treeNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return treeNode.children[item]

In [23]:
def ascendFPtree(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        ascendFPtree(node.parent, prefixPath)

def findPrefixPath(basePat, headerTable):
    # First node in linked list
    treeNode = headerTable[basePat][1]
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        ascendFPtree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next
    return condPats, frequency

def mineTree(headerTable, minSup, preFix, freqItemList):
    # Sort the items with frequency and create a list
    sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])]
    # Start with the lowest frequency
    for item in sortedItemList:
        # Pattern growth is achieved by the concatenation of suffix pattern 
        # with frequent patterns generated from conditional FP-tree
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        # Find all prefix path, constrcut conditional pattern base
        conditionalPattBase, frequency = findPrefixPath(item, headerTable)

        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructTree(conditionalPattBase, frequency, minSup)

        if newHeaderTable != None:
            # Mining recursively on the tree
            mineTree(newHeaderTable, minSup, newFreqSet, freqItemList)

In [24]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count
############################ 修改部分 ###########################
def associationRule(freqItemSet, itemSetList, minConf, minLift):    # 新增 minLift 參數
    rules = []
    for itemSet in freqItemSet:
        if len(itemSet) <= 1:
            continue
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            # lift 計算
            lift = float(itemSetSup * len(itemSetList) / getSupport(s, itemSetList) / getSupport(itemSet.difference(s), itemSetList))    
            if (confidence >= minConf) & (lift >= minLift):         # 新增 lift 的判斷並將 > 改成 >=
                rules.append([set(s), set(itemSet.difference(s)), confidence, lift])
    return rules
################################################################
def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

In [25]:
def fpgrowth(itemSetList, minSupRatio, minConf, minLift):  # 新增 minLift 參數
    frequency = getFrequencyFromList(itemSetList)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = constructTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
        return [], []
    else:
        freqItems = []
        mineTree(headerTable, minSup, set(), freqItems)
        rules = associationRule(freqItems, itemSetList, minConf, minLift)
        return freqItems, rules

In [26]:
# 自訂輸出格式
def output(rule):
    print(len(rule))
    print("⎯" * 60)
    print("{:<30} {:<15} {:<10}".format( "Association Rule", "Confidence", "Lift"))
    print("⎯" * 60)
    for r in rule:
        tmp = list(r[0])
        lhs = ", ".join(tmp)  # 左側部分
        tmp = list(r[1])
        rhs = ", ".join(tmp)  # 右側部分

        # 使用格式化字串輸出，對齊各欄位
        print("{:<30} {:<15.5f} {:<10.5f}".format(lhs + " → " + rhs, r[2], r[3]))

### MinSup = 0.0001 Conf = 0.9

In [27]:
################ Q1 ################
# 讀取資料集
with open("A1.csv", "r") as f:
    next(f)
    dataset = [line.strip().split(",") for line in f]
# 參數設定
minsup = 0.0001
minconf = 0.9
minlift = 0.0
# FP-Growth
freq, rule = fpgrowth(dataset, minsup, minconf, minlift)
# 輸出結果
output(rule)

16
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
Association Rule               Confidence      Lift      
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
fertilizer → flower soil       1.00000         2422.81250
flower soil → fertilizer       1.00000         2422.81250
nuts → prunes                  1.00000         1174.69697
prunes → nuts                  1.00000         1174.69697
bags → cling film              0.94872         496.98718 
cling film → bags              1.00000         496.98718 
film → photo                   1.00000         490.69620 
photo → film                   1.00000         490.69620 
vegetables → packaged fruit    1.00000         302.85156 
packaged fruit → vegetables    1.00000         302.85156 
red → blush wine               1.00000         246.91083 
blush wine → red               1.00000         246.91083 
fruit → vegetable juice        1.00000         74.83591  
vegetable juice → fruit        1.00000         74.83591  
rolls

In [28]:
# Frequent Item Set for Q1
# 只列印長度 2 以上
for item in freq:
    if len(item) > 1:
        print(item)

{'fertilizer', 'flower soil'}
{'nuts', 'prunes'}
{'bags', 'cling film'}
{'film', 'photo'}
{'vegetables', 'packaged fruit'}
{'red', 'blush wine'}
{'fruit', 'vegetable juice'}
{'whipped sour cream', 'buns'}
{'rolls', 'buns'}


### Minsup = 0.0001 Minlift = 90

In [29]:
################ Q2 ################
# 讀取資料集
with open("A1.csv", "r") as f:
    next(f)
    dataset = [line.strip().split(",") for line in f]
# 參數設定
minsup = 0.0001
minconf = 0
minlift = 90
# FP-Growth
freq, rule = fpgrowth(dataset, minsup, minconf, minlift)
# 輸出結果
output(rule)

12
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
Association Rule               Confidence      Lift      
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
fertilizer → flower soil       1.00000         2422.81250
flower soil → fertilizer       1.00000         2422.81250
nuts → prunes                  1.00000         1174.69697
prunes → nuts                  1.00000         1174.69697
bags → cling film              0.94872         496.98718 
cling film → bags              1.00000         496.98718 
film → photo                   1.00000         490.69620 
photo → film                   1.00000         490.69620 
vegetables → packaged fruit    1.00000         302.85156 
packaged fruit → vegetables    1.00000         302.85156 
red → blush wine               1.00000         246.91083 
blush wine → red               1.00000         246.91083 


In [30]:
# Frequent Item Set for Q1
# 只列印長度 2 以上
for item in freq:
    if len(item) > 1:
        print(item)

{'fertilizer', 'flower soil'}
{'nuts', 'prunes'}
{'bags', 'cling film'}
{'film', 'photo'}
{'vegetables', 'packaged fruit'}
{'red', 'blush wine'}
{'fruit', 'vegetable juice'}
{'whipped sour cream', 'buns'}
{'rolls', 'buns'}


### Minsup = 0.0004 , Minconf = 1, Minlift = 1000 

In [31]:
################ Q5 ################
# 讀取資料集
with open("A1.csv", "r") as f:
    next(f)
    dataset = [line.strip().split(",") for line in f]
# 參數設定
minsup = 0.0004
minconf = 1
minlift = 1000
# FP-Growth
freq, rule = fpgrowth(dataset, minsup, minconf, minlift)
# 輸出結果
output(rule)

4
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
Association Rule               Confidence      Lift      
⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
fertilizer → flower soil       1.00000         2422.81250
flower soil → fertilizer       1.00000         2422.81250
nuts → prunes                  1.00000         1174.69697
prunes → nuts                  1.00000         1174.69697
